<a href="https://colab.research.google.com/github/pranavkantgaur/gamd_sr/blob/main/sr_for_lj_potential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.10.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.10.0 on the current Colab Runtime...
2024-11-26 08:12:19 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.10/julia-1.10.0-linux-x86_64.tar.gz [168592090/168592090] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.10

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.10.0
Commit 3120989f39b (2023-12-25 18:01 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, broadwell)
  Threads: 3 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


In [9]:
using Pkg
Pkg.add("SymbolicRegression")
Pkg.add("MLJ")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
   Installed LoggingExtras ─────────────── v1.1.0
   Installed HypergeometricFunctions ───── v0.3.25
   Installed NNlib ─────────────────────── v0.9.26
   Installed ShowCases ─────────────────── v0.1.0
   Installed Accessors ─────────────────── v0.1.38
   Installed RelocatableFolders ────────── v1.0.1
   Installed StatsFuns ─────────────────── v1.3.2
   Installed ContextVariablesX ─────────── v0.1.3
   Installed CategoricalDistributions ──── v0.1.15
   Installed StaticArrays ──────────────── v1.9.8
   Installed CEnum ─────────────────────── v0.5.0
   Installed PDMats ────────────────────── v0.11.31
   Installed DelimitedFiles ────────────── v1.9.1
   Installed ConcurrentUtilities ───────── v2.4.2
   Installed InitialValues ─────────────── v0.3.1
   Installed InvertedIndices ───────────── v1.3.0
   Instal

In [16]:
import Pkg; Pkg.add("MLJBase")

   Resolving package versions...
    Updating `~/.julia/environments/v1.10/Project.toml`
  [a7f614a8] + MLJBase v1.7.0
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [57]:
function lj_potential_structure((; attr_func, rep_func), (rad, ))
  _attr_func = attr_func(rad)^-6
  _rep_func = rep_func(rad)^-3

  out = map((attr_func_i, rep_func_i) -> (attr_func_i - rep_func_i), _attr_func.x, _rep_func.x)
  return ValidVector(out, _attr_func.valid && _rep_func.valid)
end
lj_structure = TemplateStructure{(:attr_func, :rep_func)}(lj_potential_structure)

(::TemplateStructure{(:attr_func, :rep_func), typeof(lj_potential_structure), @NamedTuple{attr_func::Int64, rep_func::Int64}}) (generic function with 0 methods)

In [56]:
using LinearAlgebra

# Set parameters for Lennard-Jones potential
epsilon = 1.0  # Depth of the potential well
sigma = 1.0    # Finite distance at which the potential is zero

# Generate random radial distances
X = rand(100, 1) .* 10  # Random distances in a 3D space

# Calculate Lennard-Jones potential for each radial distance
Y = [
    4 * epsilon * ((sigma / norm(X[i, :]))^12 - (sigma / norm(X[i, :]))^6)
    for i in eachindex(axes(X, 1))
]

# Y now contains the Lennard-Jones potentials corresponding to each radial distance


100-element Vector{Float64}:
 -4.2342663500945166e-6
 -7.769415784949598e-6
 -0.12578913483097476
  5.936921461826293e7
 -0.0005384476348949936
 -0.00020643753776723449
 -0.000424749282769969
 -0.008211956926213494
 -0.00028134278269407244
  1.025058717663522e23
 -9.901873313033654e-6
 -0.002098908963473346
 -9.391888619422615e-6
  ⋮
 -8.394908892636549e-5
 -1.4295550005134211e-5
 -9.692862028183316e-5
 -6.900273056495563e-6
 -0.0002459593774332056
 -1.066925718413071e-5
 -1.1210394091627513e-5
 -0.6176838998719174
 -0.0004284239471269967
 -0.00018727719432120124
 -0.0006352153753559467
  6.753834955005477e15

In [58]:
elementwise_loss = ((x1), (y1)) -> (y1 - x1)^2

#47 (generic function with 1 method)

In [ ]:
model = SRRegressor(;
    binary_operators=(+, *, /),
    #unary_operators=(sin,),
    maxsize=15,
    elementwise_loss=elementwise_loss,
    expression_type=TemplateExpression,
    # Note - this is where we pass custom options to the expression type:
    #expression_options=(; structure),
    expression_options=(; structure = lj_structure),
)

mach = machine(model, X, Y)
fit!(mach)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter ~/.julia/packages/ProgressMeter/kVZZH/src/ProgressMeter.jl:594
Evolving for 100 iterations...  61%|████████████████████████▉                |  ETA: 0:01:46
  Info:          Full dataset evaluations per second: 5.53e+04. Press 'q' and then <enter> to stop execution early.
  Hall of Fame:                                                                                     ─────────────────────────────────────────────────────────────────────────────────────────────────── Complexity  Loss       Score      Equation                                                          2           1.040e+44  1.802e+01  ╭ attr_func = -0.00031493                                    

In [ ]:
report(mach)

In [ ]:
r = report(mach)
idx = r.best_idx
best_expr = r.equations[idx]
best_attr = get_contents(best_expr).attr_func
best_rep = get_contents(best_expr).rep_func
